Import package, data, vizualisation functions

In [ ]:
import numpy as np
import cvxopt as cvx
import picos
import itertools
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
%matplotlib inline 
matplotlib.rcParams['figure.figsize'] = (10, 10)

In [ ]:
A = cvx.matrix([[-4,-1],[-1,-2],[-2,1],[-1,-6],[1,2],[6,-2],[0,1]]).T
b = cvx.matrix([-9,-4,0,-6,11,17,4])

print 'A\n',A
print 'b\n',b

In [ ]:
def remove_midpoints(ext):
    eps = 1e-10
    n,m = ext.shape
    IJK = []
    for i,j,k in itertools.permutations(range(n),3):
        u = ext[j]-ext[i]
        v = ext[k]-ext[i]
        if u.dot(v)+np.linalg.norm(u)*np.linalg.norm(v)<=eps:
            IJK.append(i)
    return ext[list(set(range(n))-set(IJK))]

def ext_points(A,b):
    A_np = np.array(A)
    eps = 1e-10
    b_np = np.array(b)
    n,m = A_np.shape
    ext = []
    for I in itertools.combinations(range(n),m):
        xI = np.linalg.lstsq(A_np[list(I)],b_np[list(I)],rcond=-1)[0]
        if max(A_np.dot(xI)-b_np)<=eps:
            if not(ext) or min([np.linalg.norm(e.ravel()-xI.ravel()) for e in ext])>eps:
                ext.append(xI.ravel())
    ext = np.array(ext)
    return remove_midpoints(ext)

def facets(A,b,order=2):
    ext = ext_points(A,b)
    n,m = ext.shape
    A_np = np.array(A)
    b_np = np.array(b)
    eps = 1e-10
    F = []
    for I in itertools.combinations(range(n),order):
        if max(A_np.dot(ext[list(I)].T.dot([1./order]*order))-b_np.ravel())>=-eps:
            F.append(ext[list(I)]) 
    return F

def draw_poly(A,b,xc=None,r=None):
    assert A.size[1]==2 #we only handle 2D-plots
    
    ext = ext_points(A,b)
    F = facets(A,b)
    plt.scatter(ext[:,0],ext[:,1])
    for Fi in F:
        plt.plot(Fi[:,0],Fi[:,1],'r')
       
    ax = plt.gca()
    ax.set_xlim(0.5,4.5)
    ax.set_ylim(0.5,4.5)
        
def draw_ball(x0,r):
    """
    Draw the ball of radius x0 and radius r
    """
    ax = plt.gca()
    if hasattr(r,'__iter__'):#r is a sequence (normally, of length 1)
        r = r[0]
    assert len(x0)==2 #we only handle 2D-plots
    ax.add_artist(plt.Circle(x0,r))
    
def draw_ellipse(Q,x0):
    """
    Draw the ellipse {Qu+x0: |u|<=1}
    """
    lb,U = np.linalg.eigh(Q.dot(Q.T)) #eigen-decomposition of QQ'
    if lb[0]>lb[1]:
        umax = U[:,0]
        lmax = lb[0]
        umin = U[:,1]
        lmin = lb[1]
    else:
        umax = U[:,1]
        lmax = lb[1]
        umin = U[:,0]
        lmin = lb[0]
    if umax[1]>0:
        angle = np.arccos(umax[0])*180./np.pi
    else:
        angle = 360. - np.arccos(umax[0])*180./np.pi

    e = Ellipse(xy=x0,width=2*lmax**0.5,height=2*lmin**0.5,angle=angle)
    e.set_facecolor('yellow')
    ax = plt.gca()
    ax.add_artist(e)
    
    

Let us visualize the polyhedron $P$ defined by the set of inequalities $Ax\leq b$. 

We draw the ball of center $x_0=[2.5,2]^T$ and radius $r$1, which clearly isn't contained in $P$.

Then, we draw the ellipse $\{Qu+x_0: \|u\|\leq 1\}$, which is contained in $P$, but clearly not of maximal volume.


In [ ]:
x0 = [2.5,2]
r = 1
draw_ball(x0,r)
draw_poly(A,b)

Q = np.array([[0.5,0.2],[0.2,0.3]])
draw_ellipse(Q,x0)

<span style="color:blue">
Now, you have to implement (using picos) the problem of computing the largest ball contained in the polytope $\mathcal{P}=\{x: Ax\leq b\}$ **as an LP**. The decision variables for this problem are $x_0\in\mathbb{R}^n$ (the center of the ball) and $r\geq 0$ (the radius of the ball). In your code, you will need to access the $i$th row of the matrix $A$. This can be done with ``A[i,:]``.
</span style="color:blue">

<span style="color:blue">
*Hint*: For some $i\in[m]$, what is the maximum of the linear form $x\mapsto \langle a_i, x\rangle$ over the ball $B(x_0,r)$ ?
</span style="color:blue">

<span style="color:blue">
Give your answer to the *hint*-question in the following cell (with a short justification), and implement the LP in the next one.
</span style="color:blue">

*TODO write your answer here*

In [ ]:
#initialize the problem
P = picos.Problem()
m,n = A.size

#create the decision variables
#TODO

#add the constraints
#TODO

#set the objective value
#TODO

<span style="color:blue">
If you implemented the LP correctly, executing the following cell with solve the LP and display the solution.
</span style="color:blue">

In [ ]:
sol = P.solve()
print 'x_0\n',x0,'r\n',r
draw_ball(x0.value,r.value)
draw_poly(A,b)

<span style="color:blue">
The next exercise is to compute the ellipsoid of largest volume that is contained in our polytope $\mathcal{P}$. We parametrize this ellipsoid as $E=\{Q u + x_0: \|u\|\leq 1\}$, so the decision variables are $Q\in\mathbb{S}_n^+$ and $x_0\in\mathbb{R}^n$ (you can admit that $Q$ can always be chosen positive semidefinite). 
</span style="color:blue">

<span style="color:blue">
As in the previous exercise, compute (in closed-form) the maximum of the linear form  $x\mapsto \langle a_i, x \rangle$ over $E$, and deduce that the inclusion of $E$ in the halfspace $\{x: a_i^T x \leq b_i\}$ can be written as an SOCP constraint (with respect to the variables $Q$ and $x_0$).
</span style="color:blue">

*TODO write your answer here*

<span style="color:blue">
In the next cell, write an optimization problem in PICOS to compute the largest volume ellipsoid contained in $\mathcal{P}$. We recall that the volume of $P$ is a monotonic function of $\det(Q)$. In picos, maximizing the (semidefinite representable) concave function $\det^{1/n}(Q)$
can be achieved by maximizing an auxiliary variable $t$ subject to the constraint $\det^{1/n}(Q)\geq t$:
`picos.detrootn(Q)>=t`. To enter an SOCP constraint in PICOS, use the syntax: 
</span style="color:blue">

``abs(<vector affine expression>)<= <scalar affine expression>``

In [ ]:
#initialize the problem
P = picos.Problem()
m,n = A.size

#create the decision variables
#TODO

#add the constraints
#TODO

#set the objective function
#TODO

<span style="color:blue">
If you implemented the LP correctly, executing the following two cells will solve the SDP and display the solution.
</span style="color:blue">

In [ ]:
sol = P.solve()
print 'Q\n',Q,'x0\n',x0

In [ ]:
draw_ellipse(np.array(Q.value),x0.value)
draw_poly(A,b)